In [4]:
import numpy as np
from scipy.optimize import linprog
import cvxpy

In [7]:
c = np.array([2, 5, 3, 7, 7, 6])

A_ub = np.array([[1, 1, 1, 0, 0, 0], [0, 0, 0, 1, 1, 1]])
b_ub = np.array([180, 220])

A_eq = np.array([[1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1]])
b_eq = np.array([110, 150, 140])

linprog(c=c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq)

     con: array([4.39231822e-06, 6.01910284e-06, 5.61240671e-06])
     fun: 1899.9999256826582
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([7.20364821e-06, 8.82017957e-06])
  status: 0
 success: True
       x: array([1.09999995e+02, 4.43103329e-08, 6.99999980e+01, 8.98577274e-07,
       1.49999994e+02, 6.99999963e+01])

In [8]:
!pip3 install pulp
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable
# Create the model
model = LpProblem(name="sklad", sense=1)

# Initialize the decision variables
a = LpVariable(name="x1", lowBound=0, cat='Integer')
b = LpVariable(name="x2", lowBound=0, cat='Integer')
c = LpVariable(name="x3", lowBound=0, cat='Integer')
d = LpVariable(name="x4", lowBound=0, cat='Integer')
e = LpVariable(name="x5", lowBound=0, cat='Integer')
f = LpVariable(name="x6", lowBound=0, cat='Integer')

# Add the constraints to the model
model += (a + b + c == 180, "first_constraint")
model += (d + e + f == 220, "red_constraint")
model += (a + d == 110, "blue_constraint")
model += (b + e == 150, "yellow_constraint")
model += (c + f == 140, "green_constraint")

# Add the objective function to the model
model += 2 * a + 5 * b + 3 * c + 7 * d + 7 * e + 6 * f 
# Solve the problem
status = model.solve()
print(f"objective: {model.objective.value()}")

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 14.2 MB 480 kB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ilkhom/Library/Python/3.8/lib/python/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ww/67t1k6ss6bqfd8tw6ntmpfgm0000gn/T/3af3f7f78e724ee5b32aaece7d99b19e-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/ww/67t1k6ss6bqfd8tw6ntmpfgm0000gn/T/3af3f7f78e724ee5b32aaece7d99b19e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 41 RHS
At line 47 BOUNDS
At line 54 ENDATA
Problem MODEL has 5 rows, 6 columns and 12 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1900 - 0.00 sec

In [10]:
cost=np.array([
    [1000,12,10,19,8],
    [12,1000,3,7,2],
    [10,3,1000,6,20],
    [19,7,6,1000,4],
    [8,2,20,4,1000]
])
stock=np.array([1,1,1,1,1])
demand=np.array([1,1,1,1,1])
num_workers=5
num_tasks=5
c=cost.flatten()
A=[]
b=[]
for i in range(0, num_workers):
    A.append([0]*(num_tasks*i)+[1]*num_tasks+[0]*(num_tasks*(num_workers-i-1)))
    b.append(stock[i])
for j in range(0, num_tasks):
    A.append(([0] * j + [-1] + [0] * (num_tasks - j - 1)) * num_workers)
    b.append(-demand[j])
A = np.asarray(A)
b = np.asarray(b)
linprog(c=c, A_ub=A, b_ub=b)

     con: array([], dtype=float64)
     fun: 32.00000000007108
 message: 'Optimization terminated successfully.'
     nit: 8
   slack: array([-1.58761893e-13, -1.57651669e-13, -1.61204383e-13, -1.57429625e-13,
       -1.60760294e-13,  1.58095759e-13,  1.59428026e-13,  1.59650071e-13,
        1.58761893e-13,  1.60094160e-13])
  status: 0
 success: True
       x: array([1.05712516e-14, 1.83620474e-14, 3.24181434e-01, 1.08314435e-12,
       6.75818566e-01, 7.70554050e-13, 6.88028074e-15, 5.08008856e-01,
       4.91991144e-01, 2.52364073e-12, 2.93732986e-01, 4.91991144e-01,
       8.94560215e-15, 2.14275870e-01, 1.41583444e-13, 1.07350386e-12,
       5.08008856e-01, 1.67809710e-01, 8.69179879e-15, 3.24181434e-01,
       7.06267014e-01, 2.68191101e-12, 1.69659199e-13, 2.93732986e-01,
       8.22003631e-15])

In [11]:
x = cvxpy.Variable(shape=(5,5), boolean=True)
u = cvxpy.Variable(shape=5, integer=True)

from itertools import product
constraints = [
    cvxpy.sum(x, axis=0) == np.ones(5),
    cvxpy.sum(x, axis=1) == np.ones(5),
    u >= 0,
    u <= 4,
    cvxpy.sum(cvxpy.diag(x)) == 0
]

for i, j in product(range(5), range(5)):
    if i >= 0 and j >= 1 and i != j:
        constraints.append(u[i] - u[j] + 5 * x[i,j] <= 4)

d = np.array([[0, 12, 10, 19, 8], 
                [12, 0, 3, 7, 2], 
                [10, 3, 0, 6, 20], 
                [19, 7, 6, 0, 4], 
                [8, 2, 20, 4, 0]])

func = cvxpy.sum(cvxpy.multiply(x, d))
problem = cvxpy.Problem(cvxpy.Minimize(func), constraints=constraints)
result = problem.solve(solver='ECOS_BB')
print(np.round(result))

32.0


<img src="stat&basic6_img5.png" width="800px">

<h1>Задача 9.4</h1>
<img src="stat$basic_img3.jpg" width="800px">

<img src="stat$basic_img4.jpg" width="800px">

In [15]:
def f(x, y):
    return x**2 + x*y - 2*x + 3*y - 1
def grad(x, y):
    dx = 2*x + y - 2
    dy = x + 3
    return (dx, dy)

x0 = (0, 1)
vec = grad(*x0)
vec

(-1, 3)

In [17]:
A1 = [[0,1],[1,-2]]
# получил матрицу Гессе и сделал из нее обратную [0;1][1;-2]
# градиент в точке равен (-1;3)
# подставляем в формулу: x(n-1)=  (0;1) - [[0;1],[1;-2]] * (-1;3)

# Решение: Сначала нужно перемножить обратную матрицу Гессе и вектор градиента - получим вектор, а затем покомпонентно вычесть его из вектора x_n
x_n = np.array([0,1])
a = np.array(A1)
b = np.array(vec)
c = a.dot(b) # [3, -7]
x_n - c

array([-3,  8])